In [1]:
import requests
import matplotlib.pyplot as plt
import numpy as np
import shutil

In [ ]:
zoom_level = 3

long, lat = zoom_level, 0
is_long_lim = False
is_lat_lim = False

def get_url(long, lat, zoom):
    return "https://heatmap-external-c.strava.com/tiles/all/hot/" + \
            str(long) + "/" + str(lat) + "/" + str(zoom) + ".png?v=19"

def get_image_filename(long, lat, zoom):
    return str(long) + "_" + str(lat) + "_" + str(zoom) + ".png"

map_hierarchy = {}

while not is_long_lim:
    lat = 0
    is_lat_lim = False
    while not is_lat_lim:
        url = get_url(long, lat, zoom_level)
        print("Requesting", url)
        path = get_image_filename(long, lat, zoom_level)
        r = requests.get(url, stream=True)
        if r.status_code == 200:
            with open(path, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
            print("Wrote file", path)
            if long not in map_hierarchy:
                map_hierarchy[long] = []
            map_hierarchy[long].append(lat)
        else:
            print(r.status_code)
            if lat > 0:
                is_lat_lim = True
                print("Reached max latitude index:", lat)
            elif lat == 0:
                is_long_lim = True
                is_lat_lim = True
                print("Reached max longitude index:", long)
        lat += 1
    long += 1

Requesting https://heatmap-external-c.strava.com/tiles/all/hot/3/0/3.png?v=19
Wrote file 3_0_3.png
Requesting https://heatmap-external-c.strava.com/tiles/all/hot/3/1/3.png?v=19
Wrote file 3_1_3.png
Requesting https://heatmap-external-c.strava.com/tiles/all/hot/3/2/3.png?v=19
Wrote file 3_2_3.png
Requesting https://heatmap-external-c.strava.com/tiles/all/hot/3/3/3.png?v=19
Wrote file 3_3_3.png
Requesting https://heatmap-external-c.strava.com/tiles/all/hot/3/4/3.png?v=19
Wrote file 3_4_3.png
Requesting https://heatmap-external-c.strava.com/tiles/all/hot/3/5/3.png?v=19
Wrote file 3_5_3.png
Requesting https://heatmap-external-c.strava.com/tiles/all/hot/3/6/3.png?v=19
Wrote file 3_6_3.png
Requesting https://heatmap-external-c.strava.com/tiles/all/hot/3/7/3.png?v=19


In [ ]:
nrows = len(map_hierarchy.keys())
ncols = max([len(map_hierarchy[k]) for k in map_hierarchy])
fig, axs = plt.subplots(nrows, ncols)
i = 0
for k in map_hierarchy:
    j = 0
    for lat in map_hierarchy[k]:
        im = get_image_filename(k, lat, zoom_level)
        axs[i][j].imshow(im)
        j += 1
    i += 1